# Methods


## Imports


In [ ]:
import pandas as pd
import numpy as np
import natsort as ns

pd.set_option("future.no_silent_downcasting", True)

In [2]:
def read_d05(filepath: str = "example.txt"):
    with open(filepath, "r") as f:
        ranges_list, entries_list = [x.split("\n") for x in f.read().split("\n\n")]
    if entries_list[-1] == "":  # I don't like this check
        entries_list = entries_list[:-1]
    return ranges_list, entries_list

## example


In [3]:
def check_between_vectized(x, vector):
    for each in vector:
        if int(each[0]) <= int(x) <= int(each[1]):
            return x
    return None

In [15]:
ranges_list, entries_list = read_d05()

In [16]:
ranges_split = [x.split("-") for x in ranges_list]
valid_entries = [check_between_vectized(x, ranges_split) for x in entries_list]
display( np.sum([bool(x) for x in valid_entries]))

np.int64(3)

## part 1


In [6]:
input_ranges_list, input_entries_list = read_d05("input.txt")

In [7]:
input_ranges_split = [x.split("-") for x in input_ranges_list]
input_valid_entries = [
    check_between_vectized(x, input_ranges_split) for x in input_entries_list
]
display(np.sum([bool(x) for x in input_valid_entries]))

np.int64(674)

## part 2

### example


In [13]:
def compress_ranges(range_list):
    df = pd.DataFrame(range_list, columns=["start", "stop"]).astype(int)
    df = df.sort_values(["start", "stop"], ascending=True)
    df = df.drop_duplicates(subset=["start", "stop"], keep="last")
    df["interval"] = [
        pd.Interval(x[1]["start"], x[1]["stop"], closed="both") for x in df.iterrows()
    ]
    df["new_interval"] = df["interval"]
    df["new_start"] = df["start"]
    df["overlap"] = [
        x.iat[0].overlaps(x.iat[1]) if len(x) == 2 else False
        for x in df["new_interval"].rolling(window=2)
    ]
    counter = 0
    while df["overlap"].any() or counter < 10:
        df["new_start"] = np.where(df["overlap"], None, df["new_start"])  # ?
        df["new_start"] = df["new_start"].ffill()
        df = df.sort_values(["new_start", "stop"], ascending=True)
        df = df.drop_duplicates(subset=["new_start"], keep="last")
        df["new_interval"] = [
            pd.Interval(x[1]["new_start"], x[1]["stop"], closed="both")
            for x in df.iterrows()
        ]
        df["overlap"] = [
            x.iat[0].overlaps(x.iat[1]) if len(x) == 2 else False
            for x in df["new_interval"].rolling(window=2)
        ]
        counter += 1
    df["len"] = df["new_interval"].apply(lambda x: x.length + 1)
    return df["len"].sum()


compress_ranges(ranges_split)

np.int64(14)

In [14]:
df = compress_ranges(input_ranges_split)
df

np.int64(352509891817881)